In [26]:
import numpy as np
from scipy.io import loadmat
from mpl_toolkits import mplot3d
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from matplotlib.pyplot import cm
import matplotlib as mpl
import cv2
import computer_vision as cv
from icecream import ic

# %matplotlib inline
%matplotlib qt
%config InlineBackend.figure_format = 'retina'
from matplotlib import rc
rc('font', **{'family': 'serif', 'serif': ['Computer Modern']})
rc('text', usetex=True)

In [27]:
def enforce_essential(E):
    U, S, VT = np.linalg.svd(E, full_matrices=False)
    if np.linalg.det(U @ VT) < 0:
        VT = -VT
    S = [1,1,0]
    E =  U @ np.diag(S) @ VT
    return E

In [28]:
def estimate_E_DLT(img_pts_1, img_pts_2, print_svd=False):

    n = np.size(img_pts_1,1)
    M = []

    for i in range(n):

        x = img_pts_1[:,i]
        y = img_pts_2[:,i]
        m = np.outer(y, x).flatten()
        M.append([m])

    M = np.concatenate(M, 0)
    U, S, VT = np.linalg.svd(M, full_matrices=False)
    E = VT[-1,:].reshape(3,3)
    E = enforce_essential(E)

    if print_svd:
        print('\nDet(E):', np.linalg.det(E))
        for i in range(np.size(img_pts_1,1)):
            epi_const = img_pts_2[:,i].T @ E @ img_pts_1[:,i]
            print('x2^T @ E @ x1:', epi_const)

        M_approx = U @ np.diag(S) @ VT
        v = VT[-1,:] # last row of VT because optimal v should be last column of V
        Mv = M @ v
        print('||Mv||:', (Mv @ Mv)**0.5)
        print('||v||^2:', v @ v)
        print('max{||M - M_approx||}:', np.max(np.abs(M - M_approx)))
        print('S:', S)

    return E

In [29]:
def compute_epipolar_lines(F, x1, x2):
    l2 = F @ x1
    l1 = F.T @ x2
    return l1, l2

In [30]:
def compute_and_plot_lines(l, img, ax):

    col = cm.rainbow(np.linspace(0, 1, np.size(l,1)))

    for i in range(np.size(l,1)):

        a = l[0,i]
        b = l[1,i]
        c = l[2,i]

        x = np.linspace(0, np.size(img,1), 2)
        y = (-a*x - c) / b # ax + by + c = 0 ==> y = (-ax - c) / b

        ax.plot(x, y, '-', lw=3, color=col[i], alpha=0.7) #  label='Line {}'.format(i+1)

In [31]:
def plot_lines_points_and_image(img_pts, img, l, path, plt_img=False):

    fig = plt.figure(figsize=(8,6))
    ax = plt.axes()
    
    compute_and_plot_lines(l, img, ax)
    ax.plot(img_pts[0], img_pts[1], 'o', color='blue', label='Random points')

    ax.set_xlabel('$x$')
    ax.set_ylabel('$y$')
    # plt.gca().invert_yaxis()
    # ax.invert_yaxis()
    ax.set_aspect('equal')
    ax.legend(loc="upper right")
    fig.tight_layout()

    if plt_img:
        plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
    
    # ax.view_init(elev=-20, azim=-120, roll=60)
    fig.savefig(path, dpi=300)
    plt.show()


In [32]:
def point_line_distance_2D(l, p):

    D = []
    for i in range(np.size(l,1)):

        a = l[0,i]
        b = l[1,i]
        c = l[2,i]

        x = p[0,i]
        y = p[1,i]

        d = np.abs(a*x + b*y + c) / (a**2 + b**2)**0.5
        D.append([d])   
        
    D = np.concatenate(D, 0)
    return D

In [33]:
def compute_epipolar_errors(F, x1, x2):

    l1, l2 = compute_epipolar_lines(F, x1, x2)
    
    D1 = point_line_distance_2D(l1, x1)
    D2 = point_line_distance_2D(l2, x2)
    D_tot = np.concatenate((D1,D2),0)

    return D1, D2, D_tot

In [34]:
def plot_histogram(data, path):
    fig = plt.figure()
    plt.hist(data, bins=100, color='tab:blue')
    plt.xlabel('Error')
    plt.ylabel('Frequency')
    plt.xlim([0,6.5])
    fig.tight_layout()
    fig.savefig(path, dpi=300)
    plt.show()

In [35]:
def check_mean_and_std(x):
    print('\nx_mean:', np.mean(x[0,:]), '\nx_std:', np.std(x[0,:]), '\ny_mean:', np.mean(x[1,:]), '\ny_std:', np.std(x[1,:]))

In [36]:
def convert_E_to_F(E, K1, K2):
    F = np.linalg.inv(K2).T @ E @ np.linalg.inv(K1)
    return F

In [38]:
path = r'C:\Users\erikn\skola\EEN020-Computer-Vision\assignment-3'
data = r'\A3data\data'
compex1 = r'\compEx1data.mat'
compex2 = r'\compEx2data.mat'
img1 = r'\kronan1.jpg'
img2 = r'\kronan2.jpg'
report = r'\report-images'

kronan1 = cv.load_image(path+data+img1)
kronan2 = cv.load_image(path+data+img2)
K = cv.convert_mat_to_np(path+data+compex2, 'K')
K_inv = np.linalg.inv(K)
x = cv.convert_mat_to_np(path+data+compex1, 'x')
x1 = cv.dehomogenize(x[0,0])
x2 = cv.dehomogenize(x[1,0])

plt_pts = False
plt_hist = False

# x1_norm, N1 = cv.normalise_x_and_y(x1)
# x2_norm, N2 = cv.normalise_x_and_y(x2)
# x2_norm = N1 @ x2
# x1_norm = N2 @ x1

x1_norm = cv.transform_and_dehomogenize(K_inv, x1)
x2_norm = cv.transform_and_dehomogenize(K_inv, x2)

check_mean_and_std(x1_norm)
check_mean_and_std(x2_norm)

E = estimate_E_DLT(x1_norm, x2_norm, print_svd=False)
F = convert_E_to_F(E, K, K)

E = E/E[-1,-1]
F = F/F[-1,-1]

print('E:', E)
print('E:', E/E[-1,-1])
print('F:', F)

# x1 = cv.transform_and_dehomogenize(K, x1)
# x2 = cv.transform_and_dehomogenize(K, x2)

np.random.seed(0)
rand_mask = np.random.choice(np.size(x2,1), 20, replace=False)
x1_rand = x1[:,rand_mask]
x2_rand = x2[:,rand_mask]
l1, l2 = compute_epipolar_lines(F, x1_rand, x2_rand)

path1 = path+report+'/CE2_kronan1.png'
path2 = path+report+'/CE2_kronan2.png'

if plt_pts:
    plot_lines_points_and_image(x1_rand, kronan1, l1, path1, plt_img=True)
    plot_lines_points_and_image(x2_rand, kronan2, l2, path2, plt_img=True)

D1, D2, D_tot = compute_epipolar_errors(F, x1, x2)

print('\nMean distance 1:', np.mean(D1))
print('Mean distance 2:', np.mean(D2))
print('Mean distance tot:', np.mean(D_tot))

path1 = path+report+'/CE2_hist_1.png'
path2 = path+report+'/CE2_hist_2.png'
path3 = path+report+'/CE2_hist_tot.png'

if plt_hist:
    plot_histogram(D1, path1)
    plot_histogram(D2, path2)
    plot_histogram(D_tot, path3)


x_mean: -0.03388656451837208 
x_std: 0.17551215010528629 
y_mean: 0.016434083846625286 
y_std: 0.13885331387003014

x_mean: -0.07181770613872099 
x_std: 0.18966030199829914 
y_mean: 0.01497495940066028 
y_std: 0.1443538259923785
E: [[-8.88845452e+00 -1.00580666e+03  3.77078254e+02]
 [ 1.25252308e+03  7.83677160e+01 -2.44817426e+03]
 [-4.72788839e+02  2.55019170e+03  1.00000000e+00]]
E: [[-8.88845452e+00 -1.00580666e+03  3.77078254e+02]
 [ 1.25252308e+03  7.83677160e+01 -2.44817426e+03]
 [-4.72788839e+02  2.55019170e+03  1.00000000e+00]]
F: [[-3.80559180e-08 -4.29887957e-06  6.60125953e-03]
 [ 5.35336073e-06  3.34366517e-07 -3.02509203e-02]
 [-8.17379319e-03  2.98914314e-02  1.00000000e+00]]

Mean distance 1: 1.9769590042005964
Mean distance 2: 2.0837532026918084
Mean distance tot: 2.0303561034462025
